In [1]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","3TgCUfRLY3o7BXKYpWFs"),
    ca_certs="/home/isurika/downloads/elasticsearch-8.12.0/config/certs/http_ca.crt"
)
es.ping()

True

### Prepare the data

In [3]:
import pandas as pd

df = pd.read_csv("/mnt/d/University/repos/aws-chatbot/dataset/consolidated_data.csv").loc[:1500]
df.head()

,id,pattern,response
0,1,How can I run a web server on AWS?,"To run a web server on AWS, you can use Amazon..."
1,2,What AWS service should I use to host a scalab...,"For hosting scalable applications, consider us..."
2,3,How do I deploy a custom application on the cl...,"To deploy a custom application, use Amazon EC2..."
3,4,I need a dedicated environment for my applicat...,If you need dedicated resources for your appli...
4,5,What's the best way to get started with virtua...,Start with Amazon EC2. It offers a wide range ...


#### Check NA values


In [4]:
df.isna().value_counts()

id     pattern  response
False  False    False       1501
Name: count, dtype: int64

In [5]:
df.fillna("None", inplace=True)

### Convert the relevant field to Vector using BERT model

In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [7]:
df["ResponseVector"] = df["response"].apply(lambda x: model.encode(x))

In [8]:
df.head()

,id,pattern,response,ResponseVector
0,1,How can I run a web server on AWS?,"To run a web server on AWS, you can use Amazon...","[-0.00054931047, -0.05692641, -0.0006814774, 0..."
1,2,What AWS service should I use to host a scalab...,"For hosting scalable applications, consider us...","[-0.015269826, -0.012294186, -0.01100238, -0.0..."
2,3,How do I deploy a custom application on the cl...,"To deploy a custom application, use Amazon EC2...","[-0.012933653, -0.045452587, -0.0068273027, -0..."
3,4,I need a dedicated environment for my applicat...,If you need dedicated resources for your appli...,"[-0.009388072, -0.018510194, 0.0007667323, 0.0..."
4,5,What's the best way to get started with virtua...,Start with Amazon EC2. It offers a wide range ...,"[-0.059143938, -0.017358141, -0.0022847152, -0..."


In [9]:
es.ping()

True

### Create new index in ElasticSearch

In [10]:
from indexMapping import indexMapping

try:
    es.indices.create(index="all_patterns_v1", mappings=indexMapping) 
except Exception as e:
    pass

### Ingest the data into index

In [11]:
record_list = df.to_dict("records")

In [12]:
for record in record_list:
    try:
        es.index(index="all_patterns_1500", document=record, id=record["id"])
    except Exception as e:
        print(e)

In [13]:
es.count(index="all_patterns_1500")

ObjectApiResponse({'count': 1501, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

### Search the data

In [14]:
input_keyword = " Billing of Amazon EC2 systems begin and end?"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field": "ResponseVector",
    "query_vector": vector_of_input_keyword,
    "k": 1,  # Set k to 1 to get only the top result
    "num_candidates": 1500,
}

res = es.knn_search(index="all_patterns_1500", knn=query, source=["pattern", "response"])
hits = res["hits"]["hits"]


if hits:
    best_match = hits[0]
    print("Best Matching Result:")
    print("Pattern:", best_match["_source"]["pattern"])
    print("Response:", best_match["_source"]["response"])
else:
    print("No matching results found.")


Best Matching Result:
Pattern: When does billing of my Amazon EC2 systems begin and end?
Response: Billing commences when Amazon EC2 initiates the boot sequence of an AMI instance. Billing ends when the instance terminates, which could occur through a web services command, by running "shutdown -h", or through instance failure. When you stop an instance, we shut it down but don't charge hourly usage for a stopped instance, or data transfer fees, but we do charge for the storage for any Amazon EBS volumes. To learn more, visit the AWS Documentation.


/tmp/ipykernel_4759/864556165.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_patterns_1500", knn=query, source=["pattern", "response"])


In [15]:
input_keyword = "Billing of Amazon EC2 systems begin and end?"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "ResponseVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 3,
    "num_candidates" : 1500, 
}

res = es.knn_search(index="all_patterns_1500", knn=query , source=["pattern","response"])
res["hits"]["hits"]

/tmp/ipykernel_4759/1260241245.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_patterns_1500", knn=query , source=["pattern","response"])


[{'_index': 'all_patterns_1500',
  '_id': '339',
  '_score': 0.90958804,
  '_ignored': ['response.keyword'],
  '_source': {'pattern': 'When does billing of my Amazon EC2 systems begin and end?',
   'response': 'Billing commences when Amazon EC2 initiates the boot sequence of an AMI instance. Billing ends when the instance terminates, which could occur through a web services command, by running "shutdown -h", or through instance failure. When you stop an instance, we shut it down but don\'t charge hourly usage for a stopped instance, or data transfer fees, but we do charge for the storage for any Amazon EBS volumes. To learn more, visit the AWS Documentation.'}},
 {'_index': 'all_patterns_1500',
  '_id': '340',
  '_score': 0.84812176,
  '_ignored': ['response.keyword'],
  '_source': {'pattern': 'What defines billable EC2 instance usage?',
   'response': 'Instance usages are billed for any time your instances are in a "running" state. If you no longer wish to be charged for your instance

In [16]:
rdf = df.sample(frac=0.01)

In [17]:
rdf.head()

,id,pattern,response,ResponseVector
1006,1007,Can my read replicas only accept database read...,Read replicas are designed to serve read traff...,"[-0.002401607, -0.02549342, -0.01585292, 0.033..."
275,276,Why should I use Enhanced Networking?,If your applications benefit from high packet-...,"[-0.019731103, -0.011432968, -0.03366772, 0.00..."
1111,1112,I want to set up automated backups for files i...,Use S3 Object Versioning combined with S3 Life...,"[0.0029716396, 0.025104946, -0.0144500565, 0.0..."
1048,1049,What Regions does Amazon RDS Blue/Green Deploy...,Amazon RDS Blue/Green Deployments are availabl...,"[0.020765133, -0.071927235, -0.023239683, -0.0..."
1063,1064,Is Amazon RDS Optimized Writes supported on Am...,No. Amazon Aurora MySQL-Compatible Edition alr...,"[-0.051791865, 0.024290795, -0.015308269, 0.03..."


In [25]:
def search(input_keyword):
    model = SentenceTransformer('all-mpnet-base-v2')
    vector_of_input_keyword = model.encode(input_keyword)

    query = {
    "field": "DescriptionVector",
    "query_vector": vector_of_input_keyword,
    "k": 1,  # Set k to 1 to get only the top result
    "num_candidates": 1500,
}

    res = es.knn_search(index="all_patterns_1500", 
                    knn=query, 
                    source=["pattern", "response"])
    hits = res["hits"]["hits"]

    if hits:
        best_match = hits[0]
        print("Best Matching Result:")
        print("Pattern:", best_match["_source"]["pattern"])
        print("Response:", best_match["_source"]["response"])
    else:
        print("No matching results found.")

for index, row in rdf.iterrows():
    search(row["pattern"])
    # print(row["pattern"])


/tmp/ipykernel_4759/1873023959.py:12: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_patterns_1500",


No matching results found.
No matching results found.
No matching results found.
No matching results found.
No matching results found.
No matching results found.
No matching results found.
No matching results found.
No matching results found.
No matching results found.
No matching results found.
No matching results found.
No matching results found.
No matching results found.
No matching results found.
